# Bag of Words(BoW)
---

- 순서가 상관 없음
- 문서 간의 유사도를 구하는 작업에 주로 사용
  - 예) "미분","방정식" 이라는 단어의 빈도수가 높으면 수학관련 문서
- 구현 방법
  1. 각 단어에 고유한 정수 인덱스를 부여
  2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다.

## KoNLPY로 BoW 만들기

In [0]:
!pip install konlpy

In [0]:
from konlpy.tag import Okt
import re

In [0]:
room = ["바닥은 삐걱거리는 나무판자로 되어있다.",\
        "바닥은 차갑다",\
        "방 가운데에는 돌로 된 탁자가 하나 놓여있다.",\
        "방에 문이 하나 있다.",\
        "문은 나무로 되어있다.",\
        "문은 열리지 않는다.",\
        "문에는 잠금장치가 걸려있다.",\
        "방에 침대가 하나 있다.",\
        "탁자 옆에는 의자가 두 개 놓여있다.",\
        "침대 밑에는 열쇠가 있다.",\
        "방에 창문이 없다."]
okt = Okt()
token = re.sub("(\.)",""," ".join(room)) # 정규 표현식을 이용한 온점 제거
token = okt.morphs(token) # 토큰화 작업

In [10]:
word2index = {}
bow = []
for voca in token:
  if voca not in word2index.keys():
    word2index[voca] = len(word2index)
    bow.insert(len(word2index)-1,1)
  else:
    index = word2index.get(voca)
    bow[index] += 1
print(word2index)

{'바닥': 0, '은': 1, '삐걱': 2, '거리': 3, '는': 4, '나무': 5, '판자': 6, '로': 7, '되어있다': 8, '차갑다': 9, '방': 10, '가운데': 11, '에는': 12, '돌': 13, '된': 14, '탁자': 15, '가': 16, '하나': 17, '놓여있다': 18, '에': 19, '문': 20, '이': 21, '있다': 22, '열': 23, '리지': 24, '않는다': 25, '잠금장치': 26, '걸려있다': 27, '침대': 28, '옆': 29, '의자': 30, '두': 31, '개': 32, '밑': 33, '열쇠': 34, '창문': 35, '없다': 36}


In [11]:
print(bow)

[2, 4, 1, 1, 1, 2, 1, 3, 2, 1, 4, 1, 4, 1, 1, 2, 5, 3, 2, 3, 4, 2, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1]


## CountVectorizer로 BoW 만들기

- 영어만 가능
- 한국어는 조사 등의 이유로 제대로 된 BoW가 만들어지지 않는다.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ["you know I want your love. Because I love you"]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_) # 기본적으로 길이가 2 이상인 문자만 토큰으로 인식

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


## 불용어를 제거한 BoW

In [14]:
# countvectorizer에서 제공하는 불용어 사용
from sklearn.feature_extraction.text import CountVectorizer

text=["Family is not an important thing. It's everything."]
v = CountVectorizer(stop_words="english") # 직접 정의할 수도 있다.
print(v.fit_transform(text).toarray())
print(v.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


In [0]:
# NLTK에서 제공하는 불용어 사용
import nltk
nltk.download("stopwords")

In [18]:
from nltk.corpus import stopwords
sw = stopwords.words("english")
v = CountVectorizer(stop_words=sw)
print(v.fit_transform(text).toarray())
print(v.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


# 문서 단어 행렬(Document-Term Matrix, DTM)
---

- 서로 다른 문서들의 BoW를 결합.
- 문서들 간의 비교가 가능해진다.
- 행과 열을 바꿔서 TDM으로 사용하기 한다.
- 한계
  1. 희소 표현(Sparse representation)
    - 대부분의 문서 벡터 값이 0인 비효율을 보여준다.
  2. 단순 빈도 수 기반 접근
    - 불용어 또한 높은 빈도수를 가질 수 있다.

In [0]:
room = ["바닥은 삐걱거리는 나무판자로 되어있다.",\
        "바닥은 차갑다",\
        "방 가운데에는 돌로 된 탁자가 하나 놓여있다.",\
        "방에 문이 하나 있다.",\
        "문은 나무로 되어있다.",\
        "문은 열리지 않는다.",\
        "문에는 잠금장치가 걸려있다.",\
        "방에 침대가 하나 있다.",\
        "탁자 옆에는 의자가 두 개 놓여있다.",\
        "침대 밑에는 열쇠가 있다.",\
        "방에 창문이 없다.",\
        "방 안에 또 다른 방이 있다.",\
        "방과 방이 창문으로 연결되어 있다."]

In [0]:
from konlpy.tag import Kkma 
kma = Kkma()
tk_list=[re.sub("(\.)","",sen) for sen in room]
tk_list = [kma.morphs(sen) for sen in tk_list]

In [0]:
def BoW(tk):
  bow = [0 for _ in range(len(word2index))] # 앞 서 만들었던 전체 문서 word2index 활용
  for voca in tk:
    if voca in word2index.keys():
      index = word2index.get(voca)
      bow[index] += 1
  return bow

In [40]:
dtm = []
for tk in tk_list:
  dtm.append(BoW(tk))
for d in dtm:
  print(d)

[1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0

# TF-IDF(Term Frequency-Inverse Document Frequency)
---

- 단어의 빈도와 역 문서 빈도를 사용하여 DTM내의 각 단어들마다 중요한 정도를 가중치로 주는 방식.
- 문서를 d, 단어를 t, 빈도를 f 라고 가정
> - tf(d,t)
    - 특정 문서 d에서 특정 단어 t의 등장 횟수.
    - DTM에서 각 단어들이 가진 값
  - df(t)
    - 특정 단어 t가 등장한 문서의 수.
    - 문서 내의 t 빈도수와 상관 없이, t가 등장한 문서의 수만 카운트한다.
  - idf(t)
    - df(t)에 반비례하는 수
    $$ idf(d,f) = log({{n} \over {1+df(t)}}) $$
    - df를 바로 역수 취하지 않는 이유는 총 문서의 수 n이 커질수록 idf의 값이 기하급수적으로 커지기 때문이다.
- TF-IDF는 문서 전체적으로 자주 등장하는 단어는 중요도가 낮다고 판단.(the,a)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
